In [1]:
from osgeo import gdal

In [2]:
help(gdal.ReprojectImage)

Help on function ReprojectImage in module osgeo.gdal:

ReprojectImage(*args, **kwargs)
    ReprojectImage(Dataset src_ds, Dataset dst_ds, char const * src_wkt=None, char const * dst_wkt=None, GDALResampleAlg eResampleAlg, double WarpMemoryLimit=0.0, double maxerror=0.0, GDALProgressFunc callback=0, void * callback_data=None, char ** options=None) -> CPLErr



In [ ]:
# gdal.Warp('outputRaster.tif', 'inputRaster.tif', xRes=5, yRes=5)

## Raster Data Info
---

In [3]:
in_raster = '/workspace/data/Land_cover/Land_Cover/NYC_2017_LiDAR_LandCover.img'

In [4]:
ds = gdal.Open(in_raster)

In [7]:
ds_proj = ds.GetProjection()

In [10]:
ds_proj

'PROJCS["NAD83 New York State Planes, Long Island, US Foot",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",41.0333333333333],PARAMETER["standard_parallel_2",40.6666666666667],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["us_survey_feet",0.304800609601219],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

In [13]:
gt = ds.GetGeoTransform()

In [15]:
print('Top left X', gt[0])
print('Top left y', gt[3])
print('West to East (Width) Pixle Res', gt[1], ' feet')
print('North to South (height Pxile Res', gt[5], ' feet')

Top left X 912286.93
Top left y 273618.3
West to East (Width) Pixle Res 0.5
North to South (height Pxile Res -0.5


In [23]:
band = ds.GetRasterBand(1)

In [24]:
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

Band Type=Byte


In [25]:
band_min = band.GetMinimum()
band_max = band.GetMaximum()

In [29]:

if band.GetOverviewCount() > 0:
    print("Band has {} overviews".format(band.GetOverviewCount()))

if band.GetRasterColorTable():
    print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))

Band has 11 overviews


In [37]:
# CPLErr GDALRasterBand::RasterIO( GDALRWFlag eRWFlag,
#                                 int nXOff, int nYOff, int nXSize, int nYSize,
#                                 void * pData, int nBufXSize, int nBufYSize,
#                                 GDALDataType eBufType,
#                                 int nPixelSpace,
#                                 int nLineSpace )

Reading Raster Data
---

In [31]:
scanline = band.ReadRaster(xoff=0, 
                           yoff=0,
                           xsize=band.XSize, ysize=1,
                           buf_xsize=band.XSize, buf_ysize=1,
                           buf_type=gdal.GDT_Float32)

In [33]:
import struct
tuple_of_floats = struct.unpack('f' * band.XSize, scanline)

SyntaxError: invalid syntax (1394467062.py, line 1)

## Reproject
---

In [38]:
i = ds.ReadAsArray()

In [47]:
i = None

In [39]:
gt[4]

0.0

In [44]:
print(len(i[0]), len(i))

310844 314414


In [54]:
x = int(310844/4)
y = int(314414/4)

In [56]:
# x = int(len(i[0])/4)
# y = len(i)/4

In [58]:
reduc = gdal.Warp('77711_78603_Land_proj.tif', in_raster, xRes=77711, yRes=78603)

In [55]:
x, y

(77711, 78603)

In [61]:
test='/workspace/flood-risk/data-agg/77711_78603_Land_proj.tif'

In [62]:
ts = gdal.Open(test)

Warning 1: /workspace/flood-risk/data-agg/77711_78603_Land_proj.tif: TIFFReadDirectory:Bogus "StripByteCounts" field, ignoring and calculating from imagelength


In [63]:
ts.GetProjection()

'PROJCS["NAD83 New York State Planes, Long Island, US Foot",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",41.0333333333333],PARAMETER["standard_parallel_2",40.6666666666667],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

In [64]:
t = ts.ReadAsArray()

In [66]:
len(t[0])

2

In [67]:
t

array([[0, 2],
       [1, 2]], dtype=uint8)

# Trial 2
---

In [68]:
drv = gdal.GetDriverByName( 'GTiff' )
src_ds = gdal.Open(in_raster)

dst_ds = drv.Create('reproj_test_2.tif', (int(src_ds.RasterXSize/4)), (int(src_ds.RasterYSize/4)), gdal.GDT_Byte )
dst_ds.SetProjection(src_ds.GetProjectionRef())
dst_ds.SetGeoTransform(src_ds.GetGeoTransform())

gdal.ReprojectImage( src_ds, dst_ds)

cs_expected = src_ds.GetRasterBand(1).Checksum()
cs = dst_ds.GetRasterBand(1).Checksum()

# dst_ds = None

# drv.Delete( 'tmp/byte.tif' )

if cs != cs_expected:
    print('Got: ', cs)
    gdaltest.post_reason( 'got wrong checksum' )
    print('fail')
else:
    print('success')


Got:  0


NameError: name 'gdaltest' is not defined

In [70]:
ts = gdal.Open('/workspace/flood-risk/data-agg/reproj_test_2.tif')

In [71]:
ts.GetProjection()

'PROJCS["NAD83 New York State Planes, Long Island, US Foot",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",41.0333333333333],PARAMETER["standard_parallel_2",40.6666666666667],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

In [72]:
len(t[0])

2

## Trial 3 - This one Works!
---

In [88]:
reduc = gdal.Warp('reproj_test_3.tif', in_raster, xRes=2, yRes=2)

In [89]:
ts = gdal.Open('/workspace/flood-risk/data-agg/reproj_test_3.tif')

In [90]:
ts.GetProjection()

'PROJCS["NAD83 New York State Planes, Long Island, US Foot",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",40.1666666666667],PARAMETER["central_meridian",-74],PARAMETER["standard_parallel_1",41.0333333333333],PARAMETER["standard_parallel_2",40.6666666666667],PARAMETER["false_easting",984250],PARAMETER["false_northing",0],UNIT["US survey foot",0.304800609601219,AUTHORITY["EPSG","9003"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

In [91]:
i = ts.ReadAsArray()

In [92]:
len(i[0])

77711

In [93]:
len(i)

78604

In [ ]:
t

In [ ]:
sfasd